<a href="https://colab.research.google.com/github/sonmh79/VRPPD-with-robots/blob/main/Delivery_Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INFORMATION
## 1개 Seed에 대해 유전 알고리즘을 이용한 결과를 구합니다.

## 필수 엑셀 자료 형식
B1 : floors, dtype = int

B2 : robots, dtype = int

B3 : customers, dtype = int

A10 : 노드 별 수요, dtype = str('dd =' + dict)

A16~A? : travel time matrix, dtype = {(n1,n2):float(not-zero),...,}

Sheet Name : seed1, seed2, ... , seed10

In [110]:
import pandas as pd
import time
import random
from collections import OrderedDict


In [111]:
e1 = pd.read_excel('/content/drive/MyDrive/vrp_data/large_sized_210722.xlsx',sheet_name='seed6',header=None)

In [112]:
dd = e1.loc[9][0]
dd = dd.split("=")[1].strip(" ")
dd = [d.strip("{").strip(" ").strip("}") for d in dd.split(",")]
dic = {}
for d in dd:
  k = int(d.split(":")[0])
  v = int(d.split(":")[1])
  dic[k] = v
df = pd.DataFrame(dic,index=["Delivery demand"]).T
df

,Delivery demand
1,1
2,2
3,2
4,1
5,1
6,2
7,2
8,1
9,1
10,1


In [113]:
customers = int(e1.loc[2][1])
d_data = []
data = []
for d in list(e1.loc[15:15+(customers+1)*(customers)-1][0]):
  data.append(float(d.split(":")[1].rstrip(",").rstrip("}")))
  if len(data) == customers:
    d_data.append(data)
    data = []

for i in range(len(d_data)):
  d_data[i].insert(i,0)
  
d_data = pd.DataFrame(d_data)
d_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,0.00,35.69,105.83,26.69,152.69,89.69,170.83,174.83,144.55,151.69,158.69,203.55,189.69,170.69,185.69,181.69,189.55,236.55,222.55,197.55,248.83,242.55,269.69,233.55,284.55,285.83,297.69,326.69,286.83,320.83,274.69
1,35.69,0.00,87.69,83.83,93.83,110.69,166.69,167.83,177.55,144.55,145.55,185.69,173.55,140.55,198.83,224.83,236.55,249.69,194.55,253.83,217.83,213.55,258.55,274.69,295.55,248.69,271.55,293.83,315.69,351.69,334.83
2,105.83,87.69,0.00,57.55,159.55,127.69,131.69,170.69,135.55,180.69,201.83,158.69,183.83,204.83,233.55,185.83,218.69,185.83,233.69,232.55,269.55,253.83,223.69,248.83,242.83,262.69,256.55,319.55,299.69,343.69,326.55
3,26.69,83.83,57.55,0.00,116.83,132.55,109.83,114.69,171.69,169.83,179.55,190.55,163.69,212.55,229.83,181.83,199.55,191.83,252.83,217.55,207.83,262.83,265.69,279.83,256.55,236.69,237.55,294.83,301.55,344.55,314.69
4,152.69,93.83,159.55,116.83,0.00,86.55,111.83,120.83,99.69,150.69,172.55,162.69,188.55,194.55,196.83,183.83,192.83,170.55,179.69,233.55,227.69,226.69,240.55,204.69,252.55,260.55,243.83,245.55,257.69,299.55,266.55
5,89.69,110.69,127.69,132.55,86.55,0.00,124.69,125.69,131.69,104.55,153.69,160.55,202.83,194.83,153.69,178.83,228.69,162.83,220.69,174.69,195.69,253.83,181.55,245.55,255.55,294.69,224.83,308.55,266.69,311.69,280.69
6,170.83,166.69,131.69,109.83,111.83,124.69,0.00,48.55,47.83,93.69,117.55,179.69,108.69,119.69,170.83,184.55,146.55,147.55,156.69,174.69,170.83,212.55,172.69,236.55,276.83,255.83,212.55,231.83,232.83,309.83,242.69
7,174.83,167.83,170.69,114.69,120.83,125.69,48.55,0.00,26.69,40.83,161.83,146.55,125.55,141.83,148.69,128.83,194.69,211.83,148.83,189.55,194.69,228.55,182.55,252.69,244.69,217.69,205.83,269.55,239.83,277.83,269.55
8,144.55,177.55,135.55,171.69,99.69,131.69,47.83,26.69,0.00,28.69,129.55,182.69,147.55,146.55,187.83,162.83,169.55,204.69,165.83,220.83,170.55,209.69,179.83,209.83,198.55,251.69,216.69,290.83,272.69,250.83,281.83
9,151.69,144.55,180.69,169.83,150.69,104.55,93.69,40.83,28.69,0.00,162.69,134.55,172.69,148.69,125.83,156.83,143.55,149.69,195.69,181.83,216.83,175.69,237.83,206.69,259.69,208.83,217.55,235.55,230.69,299.55,303.83


In [114]:

#Initial solution construction heuristic

# 모든 노드를 다 분리해서 각각 하나의 트립으로 생각 ! 각 노드마다 가지고 있는 정보는 demand 와 trip의 트래블 타임
class Heuristic:

  """ Make Initial Solution For GA """

  def __init__(self):
    initial_trips = {}
    for i in range(1,len(self.data)+1):
      initial_trips[i]=(self.data.loc[i]["Delivery demand"],self.travel_time([0,i,0]))
      sorted_initial_trips = OrderedDict(sorted(initial_trips.items(), key = lambda x:x[1][1], reverse=True))
    feasible_trips = []
    while len(initial_trips) > 0:
        a = sorted_initial_trips.popitem(last=False)
        del initial_trips[a[0]]
        temp = []
        temp_2 = []
        if len(initial_trips) != 0:
            for i in initial_trips.keys():
                if initial_trips[i][0] + a[1][0] == 3: # robot capa = 3
                    temp.append((i, self.d_data[i][a[0]]))
                    
                elif initial_trips[i][0] + a[1][0] < 3:
                    temp_2.append((i, self.d_data[i][a[0]]))
                  
            if len(temp) != 0:
                temp.sort(key= lambda x:x[1])
                next_node = temp[0][0]
                next_node_demand = initial_trips[next_node][0]
                feasible_trips.append([a[0], next_node])
                del sorted_initial_trips[next_node]
                del initial_trips[next_node]
              
            else:
                if len(temp_2) !=0:
                    temp_2.sort(key = lambda x:x[1])
                    next_node = temp_2[0][0]
                    next_node_demand = initial_trips[next_node][0]
                    del sorted_initial_trips[next_node]
                    del initial_trips[next_node]
                    temp_3 = []
                    if len(initial_trips)!=0:
                        for i in initial_trips.keys():
                            if a[1][0] + next_node_demand + initial_trips[i][0] == 3:
                                temp_3.append((i, self.d_data[next_node][i]))
                        if len(temp_3) !=0:
                            temp_3.sort(key=lambda x:x[1])
                            next_next_node = temp_3[0][0]
                            feasible_trips.append([a[0], next_node, next_next_node])
                            del sorted_initial_trips[next_next_node]
                            del initial_trips[next_next_node]
                          
                        else:
                            feasible_trips.append([a[0], next_node])
                              
                    else:
                        feasible_trips.append([a[0], next_node])
                      
                else:
                    feasible_trips.append([a[0]])  
        else:
            feasible_trips.append([a[0]])
    res = []
    for trip in feasible_trips:
      for t in trip:
        res += [t]
    return res

  def travel_time(self,route):
    total_tt = 0
    for i in range(0, len(route)-1):
        total_tt = total_tt + d_data.loc[route[i]][route[i+1]]        
    return total_tt



In [115]:
class VRP(Heuristic):
  def __init__(self,df,d_data = d_data,capacity=3,cnum=100,mutation_prob=0.2,ev_times=100,robots=3):
    self.n = len(d_data)
    self.c = capacity
    self.data = df
    self.d_data = d_data
    self.cnum = cnum
    self.cnt = 0
    self.mutation_prob = mutation_prob
    self.ev_times = ev_times
    self.dist = 0
    self.robots = robots
    self.min_route = []
    self.info = []
    # Choose Initial Solution between Heuristic Alg. and Simple Alg.
    self.initial_input = super().__init__()
    if self._calc_route(self.initial_input) > self._calc_route(list(range(1,len(self.data)+1))):
      self.initial_input = list(range(1,len(self.data)+1))
      print("Simple Algorithm Selected for Initial Soultion !")
    else:
      print("Heuristic Algorithm Selected for Initial Soultion !")
      

    

  def _chromo(self,t=1):

    """ 1. Make Random Chromos """

    res = []
    for _ in range(t):
     res.append(random.sample(range(1,self.n),self.n-1)) 
    return res

  
  def calc_trips(self,trips):
    dist_list = []
    for trip in trips:
      d = 0
      for i in range(len(trip)-1):
        d += self.d_data.loc[trip[i]][trip[i+1]]
      dist_list.append(d)
    
    res = list(zip(dist_list,trips))
    res.sort()
    return res
  
  def split_trip(self, trip):

    """ Split Trip to Trips """

    stack = 0
    trips = []
    t = [0]
    for node in trip:
      if node == 0:
        stack +=1
      else:
        t.append(node)
      if stack == 2:
        stack = 1
        t.append(0)
        trips.append(t)
        t = [0]
    return trips
  
  def show_trip(self,route):

    """ Print the Trip of Min Route """

    stack = 0
    new_trips = [0]
    for i in range(len(route)):
      cur_demand = self.data.loc[route[i]]["Delivery demand"]
      if stack + cur_demand <= self.c:
        stack += cur_demand
        new_trips.append(route[i])
      else:
        stack= cur_demand
        new_trips = new_trips+[0]
        new_trips.append(route[i])
    new_trips += [0]
    return new_trips


  def _calc_route(self,route):

    """ 2-1-1 Calculate Distance Considering D Demands """
    stack = 0
    new_trips = [0]
    for i in range(len(route)):
      cur_demand = self.data.loc[route[i]]["Delivery demand"]
      if stack + cur_demand <= self.c:
        stack += cur_demand
        new_trips.append(route[i])
      else:
        stack= cur_demand
        new_trips = new_trips+[0]
        new_trips.append(route[i])
    new_trips += [0]

    res = 0
    for i in range(len(new_trips)-1):
      res += self.d_data.loc[new_trips[i]][new_trips[i+1]]
    return res

  def _getdist(self, chromos):

    """ 2-1 Return Distance of Each Chromo """

    dist_list = []

    for chromo in chromos:
      dist_list.append(self._calc_route(chromo))

    fit = list(zip(dist_list,chromos))
    fit.sort(key = lambda x:x[0])
    return fit

  def _rand(self,x,y):
    return int(random.uniform(x,y))

  def _crossover(self,p1,p2,hard_mode=False):

    """ 2-2-2. Crossover and Mutation """

    swap_point = self._rand(1,len(p1))
    c1,c2 = [],[]

    i = 0
    while i < swap_point:
      c1.append(p1[i])
      c2.append(p2[i])
      i+=1

    for e in p2:
      if e not in c1:
        c1.append(e)

    for e in p1:
      if e not in c2:
        c2.append(e)
    
    #mutation
    if random.uniform(0,1) <= self.mutation_prob and len(c1)>2 and not hard_mode:
      e1,e2 = random.sample(range(0,len(c1)),2)
      #target will select mutation child
      if e1 > e2:
        target = e1
      else:
        target = e2
      
      if target%2 == 0:
        c1[e1],c1[e2] = c1[e2],c1[e1]
      else:  
        c2[e1],c2[e2] = c2[e2],c2[e1]

    if hard_mode:
      e1,e2 = random.sample(range(0,len(c1)-1),2)
      c1[e1],c1[e1+1] = c1[e1+1],c1[e1]
      c2[e2],c2[e2+1] = c2[e2+1],c2[e2]

    next_gen = self._getdist([c1,c2,p1,p2])
    #print("next_gen: ",next_gen)
    c1,c2 = next_gen[0][1],next_gen[1][1]
    #print("c1,c2: ",c1,c2)
    return [c1,c2]

  
  def _select_parent(self,parents):

    """ 2-2-1. Select Parents with Roulette Algorithm  """

    fitness = [1/route[0] for route in parents] 
    sum = 0

    for f in fitness:
      sum += f
    p_index = set()

    while len(p_index)<2:
      fit = 0
      target = random.uniform(0,sum)
      for i in range(len(fitness)):
        fit += fitness[i]
        if fit > target:
          p_index.add(i)
          break  
    return list(p_index)

  
  def _make_child(self,dist_routes):

    """ 2-2. Compare Parents with Childs and Return Best Routes  """

    self.cnt += 1
    parents = dist_routes.copy()
    routes = [route[1] for route in parents]
    child = []

    # select top 100 chromos
    while parents and len(child)<self.cnum:
      selected_parents = self._select_parent(parents)
      p1,p2 = routes.pop(selected_parents[0]),routes.pop(selected_parents[1]-1)
      parents.pop(selected_parents[0]),parents.pop(selected_parents[1]-1)
      c1,c2 = self._crossover(p1,p2)
      child += c1,c2

    return child

  def evolution(self,chromo):

    """ 2. Make Childs """
    
    dist_routes = self._getdist(chromo) #(dist,routes)
    child = self._make_child(dist_routes)
    return child

  def ga(self): 

    """ 0. Start Genetic Algorithm !! """

    chromos = self._chromo(99)
    chromos.append(self.initial_input)    
    gen = self.evolution(chromos)
    self.info = self._getdist(gen)
    print("First Genenration Minimun Route: ",(self.info[0][0],self.info[0][1]))
    stack = 0
    for i in range(self.ev_times):
      gen = self.evolution(gen)
      self.info = self._getdist(gen)
      t = self.cnt
      if t % 10 == 0:
        print("{}/{}th Evolution Min Route: ".format(t,self.ev_times),(self.info[0][0],self.info[0][1]))
      
      # to avoid local minimum
      if t == 30:
        self.mutation_prob = t/100
        print(f"==================== Mutation Prob Upgrade to {self.mutation_prob} ==================== ")
      elif t == 50:
        self.mutation_prob = t/100
        print(f"==================== Mutation Prob Upgrade to {self.mutation_prob} ==================== ")
      elif t == 70:
        self.mutation_prob = t/100
        print(f"==================== Mutation Prob Upgrade to {self.mutation_prob} ==================== ")

      cur_dist = self.info[0][0]
      if self.dist == cur_dist:
        stack += 1
      else:
        stack = 0

      if stack == 30:
        self.dist,self.min_route = self.info[0][0],self.info[0][1] 
        print(f'Repeated {stack}times ------------------------------------------BREAK')  
        break
      
      self.dist,self.min_route = self.info[0][0],self.info[0][1]   
    
      
    print("----------------------------------------Finish----------------------------------------")
    print("Total Travel Time / Min Route: ",self.dist," / ",self.min_route)
    print(self.show_trip(self.info[0][1]))
#if __name__ == "__main__":
  


In [116]:
v = VRP(df,d_data,capacity=3,ev_times=200)
s= time.time()
v.ga()
e = time.time()
t = e-s
print("Total Time: ",round(t,1),"(s)")

Heuristic Algorithm Selected for Initial Soultion !
First Genenration Minimun Route:  (7502.769999999997, [27, 24, 29, 30, 26, 28, 25, 17, 22, 20, 21, 18, 23, 15, 11, 16, 19, 13, 12, 10, 14, 9, 7, 8, 6, 4, 2, 1, 5, 3])
10/200th Evolution Min Route:  (7494.0499999999965, [27, 24, 29, 30, 26, 28, 25, 17, 22, 20, 21, 18, 23, 15, 11, 16, 19, 13, 12, 10, 14, 9, 7, 8, 6, 4, 3, 1, 5, 2])
20/200th Evolution Min Route:  (7472.599999999996, [27, 24, 29, 30, 26, 28, 25, 17, 22, 20, 21, 18, 23, 15, 11, 16, 19, 13, 12, 10, 14, 9, 7, 8, 6, 4, 3, 2, 5, 1])
30/200th Evolution Min Route:  (7465.599999999996, [27, 24, 29, 30, 26, 28, 19, 17, 22, 20, 21, 18, 23, 15, 11, 16, 25, 13, 12, 10, 14, 9, 7, 8, 6, 4, 3, 2, 5, 1])
==================== Mutation Prob Upgrade to 0.3 ==================== 
40/200th Evolution Min Route:  (7465.599999999996, [27, 24, 29, 30, 26, 28, 19, 17, 22, 20, 21, 18, 23, 15, 11, 16, 25, 13, 12, 10, 14, 9, 7, 8, 6, 4, 3, 2, 5, 1])
50/200th Evolution Min Route:  (7450.4599999999955, 